# Prepare Training Data

In [1]:
import os
import cv2
import h5py
import numpy
import matplotlib.pyplot as plt

C:\Jupyter\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
Random_Crop = 30
Patch_size = 33
label_size = 21
conv_side = 6
#data path
LR_DATA_PATH = "./data/train_set/LR/"
HR_DATA_PATH='./data/train_set/HR/'
BLOCK_STEP = 14
BLOCK_SIZE = 33
%matplotlib inline

In [2]:
def prepare_crop_data():
    #get all the path for HR images
    hr_img_paths=[]
    for hr_img_file in os.listdir('./data/train_set/HR/'):
        hr_img_paths.append(os.path.join('./data/train_set/HR/',hr_img_file))
    print('Total Number of image:{}'.format(len(hr_img_paths)))
    data = []
    label = []
    for hr_img_path in hr_img_paths:
        hr_img = cv2.imread(hr_img_path)
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        shape=hr_img.shape
        lr_img_path=hr_img_path.replace('HR','LR')
        lr_img = cv2.imread(lr_img_path,cv2.IMREAD_COLOR)
        #resize image to 2 times larger
        lr_img=cv2.resize(lr_img,(shape[1],shape[0]))
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2YCrCb)
        lr_img = lr_img[:, :, 0]
        width_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) // BLOCK_STEP
        height_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) //BLOCK_STEP
        for k in range(height_num):
            for j in range(width_num):
                x = k * BLOCK_STEP
                y = j * BLOCK_STEP
                hr_patch = hr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                lr_patch = lr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                lr_patch = lr_patch.astype(float) / 255.
                hr_patch = hr_patch.astype(float) / 255.
                lr = numpy.zeros((1, Patch_size, Patch_size), dtype=numpy.double)
                hr = numpy.zeros((1, label_size, label_size), dtype=numpy.double)
                lr[0, :, :] = lr_patch
                hr[0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
                data.append(lr)
                label.append(hr)
    data = numpy.array(data, dtype=float)
    label = numpy.array(label, dtype=float)
    data=data.transpose([0,2,3,1])
    label=label.transpose([0,2,3,1])
    return data,label

In [3]:
data,label=prepare_crop_data()

Total Number of image:1500


In [4]:
data.shape

(1510997, 33, 33, 1)

In [5]:
label.shape

(1510997, 21, 21, 1)

In [7]:
def write_hdf5(data, labels, output_filename):
    x = data.astype(numpy.float32)
    y = labels.astype(numpy.float32)
    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
    print('saving completed')

In [8]:
write_hdf5(data,label,'./doc/crop_train.h5')

saving completed
